In [ ]:
%load_ext autoreload
%autoreload 2
ProjDIR = "/home/jw3514/Work/ASD_Circuits_CellType/"

try:
    os.chdir(f"{ProjDIR}/notebooks_mouse_str/")
    print(f"Current working directory: {os.getcwd()}")
except FileNotFoundError as e:
    print(f"Error: Could not change directory - {e}")
except Exception as e:
    print(f"Unexpected error: {e}")


import sys
sys.path.insert(1, ProjDIR + 'src')
from ASD_Circuits import *
#from venn import venn
import requests
import json

# Expression Level Mat construction

In [ ]:
STR_Meta = pd.read_csv(ProjDIR + 'dat/allen-mouse-exp/allen_brain_atlas_structures.csv')
STR_Meta.dropna(inplace=True, subset=["atlas_id"])
STR_Meta["atlas_id"] = STR_Meta["atlas_id"].astype(int)
#STR_Meta["parent_structure_id"] = STR_Meta["parent_structure_id"].astype(int)
STR_Meta = STR_Meta.set_index("atlas_id")
for i, row in STR_Meta.iterrows():
    STR_Meta.loc[i, "Name2"] = modify_str(row["safe_name"])


In [ ]:
STR_Meta[STR_Meta["depth"]==0]

In [ ]:
Selected_STRs = LoadList(RootDIR + 'dat/allen-mouse-exp/Structures.txt')


In [ ]:
STR_Meta_2 = STR_Meta[STR_Meta["Name2"].isin(Selected_STRs)]
STR_Meta_2 = STR_Meta_2.sort_values("Name2")
STR_Meta_2.to_csv(ProjDIR + 'dat/allen-mouse-exp/Selected_213_STRs.Meta.csv')


In [ ]:
STR_Meta_2.head(2)

In [ ]:
#Mouse2Human_Genes = pk.load(open(ProjDIR + 'dat/Mouse2Human_Entrez.pk', "rb"))

DIR = ProjDIR + 'dat/allen-mouse-exp/'
with open(DIR + 'human2mouse.0420.json', 'r') as infile:
    Human2Mouse_Genes = json.load(infile)
with open(DIR + 'mouse2sectionID.0420.json', 'r') as infile:
    Mouse2Human_Genes = {int(k): v for k, v in json.load(infile).items()}
Unlinked_AllenMouseGenes = pd.read_csv(DIR + "Allen_Unlinked_SectionIDs.0415.csv")

In [ ]:
Count1, Count2, Count3 = 0,0,0
for K, V in Mouse2Human_Genes.items():
    Count1 += 1
    #print(V)
    if len(V["humanHomo"])>0:
        Count2 += 1
    if len(V["allen_section_data_set_id"]) > 0:
        Count3 += 1
print(Count1, Count2, Count3)

In [ ]:
# Check Human Genes
Count1, Count2, Count3 = 0,0,0
for i, (entrez, v) in enumerate(Human2Mouse_Genes.items()):
    Count1 += 1
    Symbol = v["symbol"]
    mouseHomo = v["mouseHomo"]
    if len(mouseHomo) > 0:
        Count2 += 1
    g_All_Section_ID = []
    for m_symbol, m_entrez in mouseHomo:
        section_ids = Mouse2Human_Genes[m_entrez]["allen_section_data_set_id"] 
        g_All_Section_ID.extend(section_ids)
    #PRINT(g_All_Section_ID)
    if len(g_All_Section_ID) > 0:
        Count3 += 1
print(Count1, Count2, Count3)

In [ ]:
# Check Exp Level Cross Expriments
DataDIR = ProjDIR + 'dat/allen-mouse-exp/allen-mouse-brain-altas-str-unionzed/'
All_Genes = []
All_ExpEnergy = []
Counts = 0

Exp1 = []
Exp2 = []
for i, (entrez, v) in enumerate(Human2Mouse_Genes.items()):
    Symbol = v["symbol"]
    mouseHomo = v["mouseHomo"]
    g_All_Section_ID = []
    for m_symbol, m_entrez in mouseHomo:
        section_ids = Mouse2Human_Genes[m_entrez]["allen_section_data_set_id"]
        g_All_Section_ID.extend(section_ids)
    if len(g_All_Section_ID) <= 1:
        continue

    g_All_dat = []
    for section in g_All_Section_ID:
        dat_df = pd.read_csv("{}/{}.csv".format(DataDIR, section))
        _ = dat_df[dat_df["structure_id"]==997]
        if len(_) > 0:
            exp_energy = _["expression_energy"].values[0]
            g_All_dat.append(exp_energy)
        else:
            continue
    #print(entrez, Symbol, v["mouseHomo"], g_All_Section_ID)
    #print(g_All_dat)
    if len(g_All_dat)>=2:
        Exp1.append(g_All_dat[0])
        Exp2.append(g_All_dat[1])

    Counts += 1
    #if Counts > 1000:
    #    break


In [ ]:
print(len(Exp1), len(Exp2))

In [ ]:
plt.scatter(Exp1, Exp2, s=1)
plt.xlabel("Section1")
plt.ylabel("Section2")
print(pearsonr(Exp1, Exp2))

In [ ]:
# Get Root Expression
# Check Exp Level Cross Expriments
DataDIR = ProjDIR + 'dat/allen-mouse-exp/allen-mouse-brain-altas-str-unionzed/'
All_Genes = []
All_ExpEnergy = []
for i, (entrez, v) in enumerate(Human2Mouse_Genes.items()):
    Symbol = v["symbol"]
    mouseHomo = v["mouseHomo"]
    g_All_Section_ID = []
    for m_symbol, m_entrez in mouseHomo:
        section_ids = Mouse2Human_Genes[m_entrez]["allen_section_data_set_id"]
        g_All_Section_ID.extend(section_ids)
    g_All_dat = []
    for section in g_All_Section_ID:
        dat_df = pd.read_csv("{}/{}.csv".format(DataDIR, section))
        _ = dat_df[dat_df["structure_id"]==997]
        if len(_) > 0:
            exp_energy = _["expression_energy"].values[0]
            g_All_dat.append(exp_energy)
    g_avg_exp_energy = np.nanmean(g_All_dat)
    All_ExpEnergy.append(g_avg_exp_energy)
    All_Genes.append(entrez)
All_ExpEnergy = np.array(All_ExpEnergy)    
All_Genes = np.array(All_Genes)
        

In [ ]:
print(len(All_ExpEnergy), len(All_Genes), len(Human2Mouse_Genes.keys()))

In [ ]:
df = pd.DataFrame(data={"Genes": All_Genes, "EXP": All_ExpEnergy})
df = df.dropna(subset=["EXP"]).reset_index(drop=True)
df = df.sort_values("EXP", ascending=True)
df = df.reset_index(drop=True)

df["Rank"] = [x+1 for x in df.index]
df["quantile"] = df["Rank"]/len(df)
df.to_csv(ProjDIR + 'dat/allen-mouse-exp/ExpMatchFeatures.csv', index=False)
df.shape

In [ ]:
df

In [ ]:
DataDIR = ProjDIR + 'dat/allen-mouse-exp/allen-mouse-brain-altas-str-unionzed/'
All_Genes = []
All_ExpEnergy = []
for i, (entrez, v) in enumerate(Human2Mouse_Genes.items()):
    Symbol = v["symbol"]
    mouseHomo = v["mouseHomo"]
    g_All_Section_ID = []
    for m_symbol, m_entrez in mouseHomo:
        section_ids = Mouse2Human_Genes[m_entrez]["allen_section_data_set_id"]
        g_All_Section_ID.extend(section_ids)
    #print(g_All_Section_ID)
    g_All_dat = []
    if len(g_All_Section_ID) <= 0:
        continue
    for section in g_All_Section_ID:
        #print(section)
        dat_df = pd.read_csv("{}/{}.csv".format(DataDIR, section))
        #print(dat_df)
        dat = []
        for str_id, row in STR_Meta_2.iterrows():
            _id = row["id"]
            _ = dat_df[dat_df["structure_id"]==_id]
            if len(_) > 0:
                exp_energy = np.log2(1+_["expression_energy"].values[0])
            else:
                exp_energy = np.nan
                
            dat.append(exp_energy)
        g_All_dat.append(dat)
    g_All_dat = np.array(g_All_dat)
    #print(g_All_dat)
    
    g_avg_exp_energy = np.nanmean(g_All_dat, axis=0)
    #print(g_avg_exp_energy)
    
    All_Genes.append(entrez)
    All_ExpEnergy.append(g_avg_exp_energy)
    
All_ExpEnergy = np.array(All_ExpEnergy)    


In [ ]:
ExpMat = pd.DataFrame(data=All_ExpEnergy, index=All_Genes, columns=STR_Meta_2["Name2"].values)
ExpMat2 = ExpMat.dropna(axis=0, how='all', inplace=False)
print(ExpMat2.shape, ExpMat.shape)

In [ ]:
ExpMat.to_csv(ProjDIR + 'dat/allen-mouse-exp/ExpLevel.csv.gz', compression='gzip')


In [ ]:
ExpMat.head(2)

# Expression Zscore

In [ ]:
ExpMat = pd.read_csv(ProjDIR + 'dat/allen-mouse-exp/ExpLevel.csv.gz', compression='gzip', index_col=0)

In [ ]:
ExpMat.head(2)

### with QN -> Zscore

In [ ]:
ExpMat_log = ExpMat.copy()
# Apply log2(x+1) transformation column-wise, preserving NaN values
ExpMat_log = ExpMat_log.apply(lambda col: np.log2(col[pd.notnull(col)] + 1))
# Quantile Normalization
ExpMat_log_QN = quantileNormalize_withNA(ExpMat_log)

In [ ]:
df_z = []
for i in ExpMat_log_QN.index.values :
    i_z = ZscoreConverting(ExpMat_log_QN.loc[i,:].values)
    df_z.append(i_z)
ExpZscoreMat = pd.DataFrame(data=df_z, 
                            columns=ExpMat_log_QN.columns.values, 
                            index=ExpMat_log_QN.index.values)

ExpZscoreMat.to_csv(ProjDIR + 'dat/allen-mouse-exp/ExpLevel.log2.qn.Zscore.csv.gz', compression='gzip', ) 

### without QN -> Zscore

In [ ]:
df_z = []
for i in ExpMat.index.values :
    i_z = ZscoreConverting(ExpMat.loc[i,:].values)
    df_z.append(i_z)
ExpZscoreMat = pd.DataFrame(data=df_z, 
                            columns=ExpMat.columns.values, 
                            index=ExpMat.index.values)

ExpZscoreMat.to_csv(ProjDIR + 'dat/allen-mouse-exp/ExpLevel.log2.Zscore.csv.gz', compression='gzip', ) 

In [ ]:
ExpZscoreMat.head(2)

### Compute Z2 score 

# Run Z2 Score Computation in Parallel
The script `script/submit_job_run_Z2.sh` is executed to compute Z2 scores for each mouse brain section in parallel. This parallelization helps process the large dataset more efficiently.

Z2 scores are computed to:
1. compare Z1 of a gene to expression matched genes 
2. this help to both take care of expression level cross structures and genes


### Combine Z2 Matrix

In [ ]:
Z2_split_dir = RootDIR + 'dat/allen-mouse-exp/Mouse.Mean.UniformKernel.Z2scores/'
DFs = []
for file in os.listdir(Z2_split_dir):
    df = pd.read_csv(Z2_split_dir + file, index_col = 0)
    DFs.append(df)
Z2_MAt = pd.concat(DFs)
Z2_MAt.to_csv(RootDIR +"dat/AllenMouseBrain_Z2bias.csv")

In [ ]:
# Z2 Mat with neuron norm

Z2_split_dir = RootDIR + "dat/allen-mouse-exp/Mouse.Mean.UniformKernel.Z2scores.neuronorm/"
DFs = []
for file in os.listdir(Z2_split_dir):
    df = pd.read_csv(Z2_split_dir + file, index_col = 0)
    DFs.append(df)
Z2_MAt = pd.concat(DFs)
Z2_MAt.to_csv(RootDIR + "dat/allen-mouse-exp/AllenMouseBrain_Z2bias.neuronorm.csv")

In [ ]:
Z2_split_dir = RootDIR + "dat/allen-mouse-exp/Mouse.Mean.UniformKernel.Z2scores.neuro2glia/"
DFs = []
for file in os.listdir(Z2_split_dir):
    df = pd.read_csv(Z2_split_dir + file, index_col = 0)
    DFs.append(df)
Z2_MAt = pd.concat(DFs)
Z2_MAt.to_csv(RootDIR + "dat/allen-mouse-exp/AllenMouseBrain_Z2bias.neuro2glia.csv")